In [1]:

import EegFunctions
import importlib
importlib.reload(EegFunctions)
from EegFunctions import *
%matplotlib qt

In [2]:
class Config:
    """Configuration settings for P300 analysis"""
    
    # File paths
    
    DATA_FILE =  r'C:\Users\Keerti\Downloads\MySolution\PlayBooks\data\1_MMN_preprocessed_interp.set'
    DEVICE = 'unicorn'  # 'muse' or 'unicorn'
    SAMPLING_RATE = 250
    CHANNELS = ['TP9', 'AF7', 'AF8', 'TP10'] if DEVICE == 'muse' else ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8']
    ANALYSIS_CHANNELS = ['TP9', 'TP10'] if DEVICE == 'muse' else [ 'Pz','Cz']
    
    EPOCH_TMIN = -0.1
    EPOCH_TMAX = 0.6
    BASELINE = (-0.1, 0)
    HIGH_PASS = 1.0  # High-pass filter cutoff frequency
    LOW_PASS = 30.0  # Low-pass filter cutoff frequency
    

In [3]:
raw,df = create_mne_raw(Config.DATA_FILE)
events,event_ids,stimulus_count = extract_events(raw=raw)
Config.EPOCH_TMAX = 0.8
Config.EPOCH_TMIN = -0.2
Config.BASELINE = (-0.2,0)
Config.EVENT_DICT = event_ids

montage = mne.channels.make_standard_montage('standard_1020')
eog_channels = ['HEOG-left', 'HEOG-right', 'VEOG-lower', 'HEOG-bipolar', 'VEOG-bipolar']
channel_types = {ch: 'eog' for ch in eog_channels}
raw.set_channel_types(channel_types)
raw.set_montage(montage, match_case=False)


epochs = create_epochs(raw, events,Config)
bin_1_idx, bin_2_idx = create_bins(events)
print(f"Bin 1 (Deviant preceded by Standard) trials: {len(bin_1_idx)}")
print(f"Bin 2 (Standard preceded by Standard) trials: {len(bin_2_idx)}")
epochs_bin1 = epochs[bin_1_idx]  # Deviant preceded by standard
epochs_bin2 = epochs[bin_2_idx]
print(len(epochs_bin1), len(epochs_bin2))

=== CREATING MNE OBJECT ===


c:\Users\Keerti\Downloads\MySolution\PlayBooks\EegFunctions.py:35: RuntimeWarning: Estimated head radius (11.8 cm) is above the 99th percentile for adult head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(path, preload=True)


=== EXTRACTING EVENTS ===

Found 1002 annotations:
  1: 2 occurrences
  70: 200 occurrences
  180: 15 occurrences
  80: 785 occurrences
Used Annotations descriptions: [np.str_('1'), np.str_('180'), np.str_('70'), np.str_('80')]
=== CREATING EPOCHS ===
Not setting metadata
1002 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1002 events and 257 original time points ...
0 bad epochs dropped
1: 2
180: 15
70: 200
80: 785
Created epochs - 1: 2, 180: 15, 70: 200, 80: 785
Bin 1 (Deviant preceded by Standard) trials: 200
Bin 2 (Standard preceded by Standard) trials: 585
200 585


In [ ]:
def moving_window_peak_to_peak_artifact_detection(epochs, channel_idx=32, 
                                                 window_width=0.2, window_step=0.1, 
                                                 threshold=100e-6):
   
    ch_idx = channel_idx 
    print(f"Using channel index {ch_idx} ({epochs.ch_names[ch_idx]}) for artifact detection.")
    sfreq = epochs.info['sfreq']
    window_samples = int(window_width * sfreq)
    step_samples = int(window_step * sfreq)
    data = epochs.get_data()[:, ch_idx, :] # Shape: (n_epochs, n_times)
    n_epochs, n_times = data.shape
    
    bad_epochs = [None] * n_epochs
    
    # Process each epoch
    for epoch_idx in range(n_epochs):
        epoch_data = data[epoch_idx, :]
        
        # Apply moving window
        for start_idx in range(0, n_times - window_samples + 1, step_samples):
            end_idx = start_idx + window_samples
            window_data = epoch_data[start_idx:end_idx]
            
            # Calculate peak-to-peak amplitude in this window
            peak_to_peak = np.max(window_data) - np.min(window_data)
            
            # Check if threshold is exceeded
            if peak_to_peak > threshold:
                bad_epochs[epoch_idx] = epoch_idx
                break  # No need to check more windows in this epoch
    
    return bad_epochs
epochs_filter = epochs.copy()
Config.LOW_PASS = 30
epochs_filter.filter(l_freq=Config.HIGH_PASS,
        h_freq=Config.LOW_PASS, 
    )

thresholds_uv = [ 150,170]
thrsh = 100
print(f"\nUsing threshold: {thrsh} µV")
bad_epochs = moving_window_peak_to_peak_artifact_detection(epochs_filter, 
                                                            channel_idx=epochs_filter.ch_names.index('VEOG-bipolar'),
                                                            window_width=0.2, window_step=0.1, 
                                                            threshold=thrsh * 1e-6)
rejection_rate = np.where(bad_epochs)[0].shape[0] / len(epochs_filter) * 100
print(f"Trials flagged for rejection: {np.where(bad_epochs)[0].shape[0]} ({rejection_rate:.1f}%)")


Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 845 samples (3.301 s)



C:\Users\Keerti\AppData\Local\Temp\ipykernel_15152\2911719732.py:35: RuntimeWarning: filter_length (845) is longer than the signal (257), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filter.filter(l_freq=Config.HIGH_PASS,



Using threshold: 100 µV
Using channel index 32 (VEOG-bipolar) for artifact detection.
Trials flagged for rejection: 278 (27.7%)


In [25]:
def moving_window_step_artifact_detection(epochs, channel_name="VEOG-lower",window_size=0.2, step_size=0.01, threshold=100):
    sfreq = epochs.info['sfreq']
    win_samp = int(window_size * sfreq)
    step_samp = int(step_size * sfreq)

    # Pick an EOG channel (e.g., VEOG-lower)
    eog_data = epochs.copy().pick([channel_name]).get_data() * 1e6  # µV
    n_epochs = eog_data.shape[0]
    bad_epochs = [None] * n_epochs

    for i, epoch in enumerate(eog_data):
        max_diff = 0
        for start in range(0, epoch.shape[1] - win_samp, step_samp):
            half = win_samp // 2
            mean1 = epoch[0, start:start+half].mean()
            mean2 = epoch[0, start+half:start+win_samp].mean()
            diff = abs(mean2 - mean1)
            max_diff = max(max_diff, diff)
        if max_diff > threshold:
            bad_epochs[i] = i 
    return bad_epochs
bad_epochs = moving_window_step_artifact_detection(epochs, channel_name="VEOG-lower", window_size=0.2, step_size=0.01, threshold=50)


In [26]:
epochs.drop_log = [None] * len(epochs)
for i, is_bad in enumerate(bad_epochs):
    if is_bad:
        epochs.drop_log[i] = "BLINK_ARTIFACT"

print(f"\nMarked {np.where(epochs.drop_log)[0].shape[0]} epochs for rejection due to blinks; {np.where(bad_epochs)[0].shape[0] / len(epochs) * 100:.1f}% of total epochs.")


Marked 73 epochs for rejection due to blinks; 7.3% of total epochs.


In [27]:
plot_bad_epochs(epochs,bad_epochs,'VEOG-bipolar',n_epochs_to_plot=15,title="Blink Artifact Detection")

Using matplotlib as 2D backend.
